In [13]:
import pandas as pd

#数据处理
#将终点相同的cell合并，一列终点cell，一列起点cell的字符串集合
#数据有问题，需要先将 ' '换成',', 用txt 替换可以，
#给数据加列名 ，user，start，end，time1，time2
data = pd.read_csv(r'..\data\od.csv',usecols=['start','end']) 
data['start'] = data['start'].astype(str)
data['end'] = data['end'].astype(str)
data['start'] = data['start'].apply(lambda x:' '+ str(x))
data = data[['start','end']]
data = data.groupby(by='end').sum()
data['start'] = data['start'].apply(lambda x: x[1:])
print(data.head())
print(data.info())
data_samples = data['start']

                                                   start
end                                                     
10000  10561 12699 10178 11002 11443 11852 12244 1020...
10001  12225 10803 10129 10663 10155 11794 10652 1042...
10002  10416 12222 10762 10499 10416 10416 10416 1046...
10003  10356 12338 12307 11306 11896 10347 10334 1018...
10004  10329 12337 10365 10331 10330 12730 12043 1035...
<class 'pandas.core.frame.DataFrame'>
Index: 2139 entries, 10000 to 13402
Data columns (total 1 columns):
start    2139 non-null object
dtypes: object(1)
memory usage: 33.4+ KB
None


In [17]:
from time import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

n_samples = 2000
n_features = 1000
n_components = 10
n_top_words = 20 # 显示主题的前n_top_words个cell

# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer() 
# tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
#                                 max_features=n_features,
#                                 stop_words='english')
t0 = time()
tf = tf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))
print()

Extracting tf features for LDA...
done in 2.906s.



In [18]:
lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))



done in 9.940s.


In [28]:
def print_top_words(model, feature_names, n_top_words):
    f = open(r'../result/lad.csv','w') 
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d:," % topic_idx
        message += ",".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        message +="\n"
        print(message)
        f.write(message)
    f.close()
    print()
print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)


Topics in LDA model:
Topic #0:,12348,11827,12252,11830,11831,12240,11829,12362,12244,11889,11822,12440,11835,12421,12242,11856,11890,11891,11804,12342

Topic #1:,13206,13046,12497,12507,12525,13210,13402,12761,12508,12783,13383,13379,12757,12745,13387,13392,13130,12760,13103,12753

Topic #2:,11957,12416,11971,11815,12214,11953,12210,12135,11961,11810,12434,11809,11787,12391,12428,12326,12127,11940,12417,12316

Topic #3:,10533,13024,10449,10064,10428,10412,10466,10431,12372,10467,13354,10768,13362,13022,10089,10437,10448,13029,10018,13017

Topic #4:,12231,12237,12229,12233,12234,10085,12225,12224,12230,10067,12206,12316,12254,12305,11789,10086,10804,10092,11796,10658

Topic #5:,11713,11466,12334,12097,12070,12084,12079,11761,12439,11899,11772,11843,11467,11411,11408,12115,11771,11329,12094,11334

Topic #6:,12501,12686,10107,12681,12699,10293,12691,10085,10098,10297,10175,13272,10181,10011,12640,12646,10193,10093,10686,10179

Topic #7:,12597,12582,13278,10964,11352,12727,11340,12732,114